In [1]:
%pylab inline
import time
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [2]:
def get_box(key):
    for r_idx, r in enumerate(ROW):
        for c_idx, c in enumerate(COL):
            if (key[0] == r) and (key[-1] == c):    
                return boxes[r_idx][c_idx]

def check_cols(n, col, board):
    for key in board.keys():
        if key[-1] == col:
            if board[key] == n:
                return False
    return True

def check_rows(n, row, board):
    for key in board.keys():
        if key[0] == row:
            if board[key] == n:
                return False
    return True

def check_box(n, box):
    global box_to_vals
    if n in box_to_vals[box]:
        return False
    return True
    

In [3]:
def is_legal(n, coord, board):
    
    return (check_cols(n, coord[-1], board) and check_rows(n, coord[0], board) \
            and check_box(n, get_box(coord)) and (n>0) and (n<10))\
            and (board[coord]==0)

In [38]:
def is_complete(board):
    return (not 0 in board.values())

def all_legal_vals(coord, board):
    legal_vals = []
    for n in range(1,10):
        if is_legal(n, coord, board):
            legal_vals.append(n)
    return legal_vals

def get_mrv(open_coords, board):
    """
    Returns the unfilled box coordinate (coord) with minimum remaining [legal] values (MRV) 
    and the corresponding values (available domain)
    """
    #Could this be improved?
    possible_moves = [(all_legal_vals(coord, board), coord) for coord in open_coords]
    n_moves_per_box = [len(moves[0]) for moves in possible_moves]
    idx = np.argmin(n_moves_per_box)
    return open_coords[idx], possible_moves[idx][0]

def get_mrv_improved():
    #Sort list of available moves from global moveset dict.
    #return minimum.
    to_return = sorted([(len(v), v, k) for k,v in moveset.items() if v])
    if to_return:
        return to_return[0]
    else:
        return


def solve_mrv(board):
    b = board.copy()

    if is_complete(b):
        return b
    
    '''
    get coordinates of available spaces to fill
    '''
    mrv_result = get_mrv_improved()

    if mrv_result:
        _, vals, coord = mrv_result
        for value in vals:
            row_coords = [i+j for i,j in product(coord[0], COL)]
            col_coords = [j+i for i,j in product(coord[1], ROW)]
            box_coords = [i for i in box_to_coords[get_box(coord)]]
            impacted = set(row_coords+col_coords+box_coords)
            impacted.remove(coord)

            b[coord] = value
            old_moveset_values = moveset[coord]
            moveset[coord] = set()

            removed = []
            for c in impacted:
                if value in moveset[c]:
                    moveset[c].remove(value)
                    removed.append(c)

            result = solve_mrv(b)
            if result:
                return result
            else:
                b[coord] = 0
                moveset[coord] = old_moveset_values
                for c in removed:
                    moveset[c].add(value)

    return False

In [39]:
ROW = "ABCDEFGHI"
COL = "123456789"

def to_np(inp):
    bd = np.array(list(inp.values())).reshape((9,9))
    return bd

def to_dict(line):
    global ROW
    global COL
    return {ROW[r] + COL[c]: int(line[9*r+c]) for r in range(9) for c in range(9)}

def parse_grid(board):
    global box_to_vals
    for key in board.keys():
        val = board[key]
        box = get_box(key)
        if box_to_vals.get(box):
            box_to_vals[box].append(val)
        else:
            box_to_vals[box] = [val]



In [40]:
from itertools import product

sudokus = []
with open('starter/sudokus_start.txt', 'r+') as f:
    sudokus = [to_dict(i.strip()) for i in f.readlines()]

for i in range(30):
    boxes = [[(i//3,j//3) for i in range(9)] for j in range(9)]
    box_to_vals = dict()
    solution_data = dict()

    board = sudokus[i]
    print(to_np(board))
    parse_grid(board)

    coords = [i+j for i,j in product(ROW, COL)]
    box_to_coords = dict()
    for coord in board.keys():
        box = get_box(coord)
        if not box_to_coords.get(box):
            box_to_coords[box] = []
        box_to_coords[box].append(coord)

    moveset = {coord:set(all_legal_vals(coord, board)) for coord, val in board.items()}
    print(to_np(solve_mrv(board)))

[[0 0 3 0 2 0 6 0 0]
 [9 0 0 3 0 5 0 0 1]
 [0 0 1 8 0 6 4 0 0]
 [0 0 8 1 0 2 9 0 0]
 [7 0 0 0 0 0 0 0 8]
 [0 0 6 7 0 8 2 0 0]
 [0 0 2 6 0 9 5 0 0]
 [8 0 0 2 0 3 0 0 9]
 [0 0 5 0 1 0 3 0 0]]
[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [2 5 1 8 7 6 4 9 3]
 [5 4 8 1 3 2 9 7 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 7 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 2]]
[[0 0 0 2 6 0 7 0 1]
 [6 8 0 0 7 0 0 9 0]
 [1 9 0 0 0 4 5 0 0]
 [8 2 0 1 0 0 0 4 0]
 [0 0 4 6 0 2 9 0 0]
 [0 5 0 0 0 3 0 2 8]
 [0 0 9 3 0 0 0 7 4]
 [0 4 0 0 5 0 0 3 6]
 [7 0 3 0 1 8 0 0 0]]
[[4 3 5 2 6 9 7 8 1]
 [6 8 2 5 7 1 4 9 3]
 [1 9 7 8 3 4 5 6 2]
 [8 2 6 1 9 5 3 4 7]
 [3 7 4 6 8 2 9 1 5]
 [9 5 1 7 4 3 6 2 8]
 [5 1 9 3 2 6 8 7 4]
 [2 4 8 9 5 7 1 3 6]
 [7 6 3 4 1 8 2 5 9]]
[[0 0 0 1 0 0 7 0 2]
 [0 3 0 9 5 0 0 0 0]
 [0 0 1 0 0 2 0 0 3]
 [5 9 0 0 0 0 3 0 1]
 [0 2 0 0 0 0 0 7 0]
 [7 0 3 0 0 0 0 9 8]
 [8 0 0 2 0 0 1 0 0]
 [0 0 0 0 8 5 0 6 0]
 [6 0 5 0 0 9 0 0 0]]
[[9 5 6 1 3 8 7 4 2]
 [2 3 7 9 5 4 8 1 6]
 [4 8 1 

In [ ]:
'''
Change Code to work with above cell.
'''

'''
run = False

if run:
    for idx, board in tqdm(enumerate(sudokus[:2])):
        #Reset pertinent globals:
        box_to_vals = dict()

        #board = sudokus[16]

        #Prep grid reference:
        for key in board.keys():
            val = board[key]
            box = get_box(key)
            if box_to_vals.get(box):
                box_to_vals[box].append(val)
            else:
                box_to_vals[box] = [val]

        start = time.time()
        solution = solve_mrv(board)
        end = time.time()

        time_to_solve = end-start

        solution_data[idx] = {'solution':solution, 'time':time_to_solve}

        if solution:
            print(to_np(solution))
        else:
            print('No solution found.')

        print(f'{(time_to_solve)} sec')
'''